In [122]:
import sys

sys.version

'3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]'

In [123]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [164]:
print(webdriver.__version__)


4.3.0


In [160]:
import pandas as pd    # csv
from selenium import webdriver
from selenium.webdriver.chrome.options import *
from bs4 import *
import time
import re
import base64    # encoder

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [125]:
# 웹페이지 접속

In [217]:
# webdriver = 웹페이지를 제어하기 위한 모듈 >> 웹 그자체가 됨!
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys
# time = 컴퓨터한테 쉬는 시간을 부여, 컴퓨터한테 클릭 텀 시간주기
import time

## 크롬 드라이버 실행
#  wb.Chrome() = 컴퓨터가 크롬창을 씀
driver = wb.Chrome()
# 컴퓨터에서 어디를 들어갈지 url주소를 알려줌
url = "https://www.modoodoc.com/"
driver.get(url)

# 로그인
id = 'random1234@random.com'
password = 'random1234'

driver.find_element('xpath', '/html/body/nav/div/div/ul[2]/ul/li[3]/a').click() #id 속성으로 접근

user_email = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[1]')
user_email.send_keys(id)
user_password = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[2]')
user_password.send_keys(password)

# 로그인 버튼 클릭
driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/button').click()
time.sleep(3)

# 피부과 탭으로 이동
driver.find_element("xpath", "/html/body/div[2]/div/table/tbody/tr[1]/td[3]/a").click()    #링크가 달려 있는 텍스트로 접근
time.sleep(1)



In [218]:
# 첫번째 병원 탭으로 이동
driver.find_element('xpath', '/html/body/div[3]/div/div/div[1]/div[3]/div[5]/a/div/div[2]').click()

In [216]:
# 크롤링 후 리스트 삽입

data = []
data_unit = []

for i in range(2, 16):
    try:
        # 병원 이름 크롤링
        hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

        # 받은 치료 이름 크롤링
        treatment = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{i}]/div/div[2]/div[2]').text
        
        # 받은 치료 명시 안된 부분 처리
        if treatment[0] != '받':
            continue
        treatment = treatment[8:]

        # 리뷰 내용 크롤링
        review = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{i}]/div/div[2]/div[3]').text

        # 크롤링 데이터 리스트 삽입
        data_unit = [hospital, treatment, review]
        data.append(data_unit)

    except:
        pass

data

[['뮤즈의원 강남점',
  '보톡스',
  '집에서 가까워서 미간주름 보톡스 맞으러 갔는데 효과는거의 없었어요,,맞기 전이랑 후랑 차이를 못느꼈어요'],
 ['뮤즈의원 강남점',
  '제모',
  '레이저 제모 5회권을 끊어놨는데 예전 폰에 결제 문자가 와서 인증이 안 되네요ㅠ 직원분들도 친절하신 편이고 의사쌤이랑 저랑 이름이 같아서 웃으면서 치료받았던 기억이 납니당:) 현재 3회차까지 시술을 받아서 아직 완전히 모근이 없어지진 않았지만 차차 없어질거라서 기대하고 계속 다닙니다:)\n×종아리 보톡스도 300유닛 맞았는데 이건 효과가 없는 게 아닌가 하는 생각이 듭니다;; 18만인가 19만 얼마였던 거 같은데ㅠ 그건 아빠카드로 긁어서 ㅠㅠㅠ,,;;; 1달이 됐는데도ㅠ']]

In [258]:
# 페이지 이동

count_review = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[2]/div/div[1]/div[1]').text[7:]

count_page = 0

383
3


In [219]:
data = []
data_unit = []

# 병원 이름 크롤링
hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

# 받은 치료 이름 크롤링
treatment = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[2]/div/div[2]/div[2]').text[8:]

# 리뷰 내용 크롤링
review = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[2]/div/div[2]/div[3]').text

# 크롤링 데이터 리스트 삽입
data_unit = [hospital, treatment, review]
data.append(data_unit)
data

[['뮤즈의원 강남점',
  '필러',
  '입꼬리 필러 맞으러 갔어요 전에 턱보톡스 맞았던 곳이라 그냥 가봤는데 필러놓아주는 선생님도 친절하셨습니다. 아직 멍들고 부어있지만 뚱한 인상이었던 입꼬리가 달라져서 마음에 들어요.']]

In [179]:
# 병원 이름 크롤링
hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

# 받은 치료 이름 크롤링
treatment = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[3]/div/div[2]/div[2]').text[8:]

# 리뷰 내용 크롤링
review = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[3]/div/div[2]/div[3]').text

# 크롤링 데이터 리스트 삽입
data_unit = [hospital, treatment, review]
data.append(data_unit)
data

[['뮤즈의원 강남점',
  '필러',
  '입꼬리 필러 맞으러 갔어요 전에 턱보톡스 맞았던 곳이라 그냥 가봤는데 필러놓아주는 선생님도 친절하셨습니다. 아직 멍들고 부어있지만 뚱한 인상이었던 입꼬리가 달라져서 마음에 들어요.'],
 ['뮤즈의원 강남점',
  '제모',
  '이벤트 기간에 인중, 겨드랑이, 다리 전체 제모를 받았습니다. 직원들이랑 의사 선생님 모두 친절하세요. 손님들이 많아서 기다리는 시간이 좀 있었어요. 제모 효과는 괜찮은 편입니다.']]

In [194]:
# 병원 이름 크롤링
hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

# 받은 치료 이름 크롤링
treatment = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[2]').text[8:]

# 리뷰 내용 크롤링
review = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[3]').text

# 크롤링 데이터 리스트 삽입
data_unit = [hospital, treatment, review]
data.append(data_unit)
data

[['뮤즈의원 강남점',
  '필러',
  '입꼬리 필러 맞으러 갔어요 전에 턱보톡스 맞았던 곳이라 그냥 가봤는데 필러놓아주는 선생님도 친절하셨습니다. 아직 멍들고 부어있지만 뚱한 인상이었던 입꼬리가 달라져서 마음에 들어요.'],
 ['뮤즈의원 강남점',
  '제모',
  '이벤트 기간에 인중, 겨드랑이, 다리 전체 제모를 받았습니다. 직원들이랑 의사 선생님 모두 친절하세요. 손님들이 많아서 기다리는 시간이 좀 있었어요. 제모 효과는 괜찮은 편입니다.'],
 ['뮤즈의원 강남점',
  '보톡스',
  '처음 가봤는데\n선샘님과 기타 간호사님들 코디분들 다 친절하시고\n가격도좋고\n효과도 있는것같아서 재방문의사 있어요\n\n다만 2층에서 데스크있는분들중 바쁜건알겠지만\n본인기분나쁜걸 표현하시는건지 그렇게 말하시면 안되는거 아닌가 싶어요~\n주의 부탁드리고\n병원 시술효과는 좋은편이라 자주 방문할것 같습니다'],
 ['뮤즈의원 강남점',
  ' 알아보고 병원마다 가격 비교해본 결과 당시에 5회 기준 가장 저렴해서 뮤즈의원을 선택하였고 헤어라인을 포함 겨드랑이와 인중 제모도 진행하였습니다 레이저 제모 특성상 다섯번 한다고 효과는 바로 나타나지 않지만 병원 시설이 매우 쾌적하였고 데스트 안내 직원들도 모두 친절하게 응대해주셔서 불편함 없이 다닐수 있었습니다!',
  '치료 가격정보'],
 ['뮤즈의원 강남점',
  '시너지레이저',
  '한줄평: 사람 많아요\n\nMpx 시너지 레이저 3회권 끊어서 받았는데 어떤 의사한테 받느냐에 따라 시간도 효과도 달랐어요 일단 바람이 나오는데 코부근 할때 숨참고 하는게 힘들어요\n\n병원리뷰:\n사람 많아서 도떼기 시장입니다 2시간 기다린적도 있네요 예약해도 30분 기다린적도 있고. 직원들은 데스크 대체적으로 친절 시설 좋지만 이용객이 많다보니 세면대 지저분'],
 ['뮤즈의원 강남점',
  '받은',
  '처음 가봤는데\n선샘님과 기타 간호사님들 코디분들 다 친절하시고\n가격도좋고\n효과도 있는것같아서 재방문의사 있어요\n\n다

In [131]:
# parsing을 위한 함수를 생성

In [132]:
# def remove_html_tags(data):
#     p = re.compile(r'<.*?>')
#     return p.sub(' ', str(data))

# def get_crawl(URL):
#     response = driver.get(URL)
#     html = driver.page_source
#     soup7 = BeautifulSoup(html, 'html.parser')
#     ex_id_divs = soup7.find('div', {'id': 'view_content'})
#     crawl_data = remove_html_tags(ex_id_divs)
#     return crawl_data